# Generating new words

### Load all necessary items

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from app.ml_models.rnn.rnn_model import RNNAnna
from app.ml_models.rnn.generate import generate_word
from app.ml_models.rnn.data_tools import WordLevelDataset

Database URI: sqlite:////home/anna/Documents/prog/dutchlm/straattaal/app.db


In [4]:
import os
from collections import Counter
import torch
from app.ml_models.rnn.loaded_rnn_model import load_model
m, v = load_model('2021_straattaal_epoch100.pt', extra_path=['..'], device='cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
existing = WordLevelDataset(
        prefix="../data",
        filename_datasets=["straattaal.txt", "dutch.txt"],
        vocabulary=v,
    ).all_words_to_set()

### Keep track of words that are copied

In [6]:
copy_traindata = Counter()
def generate_words(amt = 10, temperature=0.3, do_print=True):
    copied = 0
    for i in range(amt):
        word = generate_word(m, v, temperature=temperature, start_letter='random')
        #distance = min([(editdistance.distance(a,word),a) for a in existing])
        if word not in existing:
            if do_print:
                print(word)
        else:
            copy_traindata[word] += 1
            copied +=1
        
    print(f'Copied words for temperature={temperature}', copied/amt)

In [7]:
generate_words(20, 0.35,True)

skippa
hapi
uitg
moruj
ragie
granka
trala
yottie
zibren
wkeren
Copied words for temperature=0.35 0.5


In [8]:
for t in [0.001, 0.1,0.3,0.4,0.6,1.0]:
    generate_words(100,t,False)

Copied words for temperature=0.001 0.76
Copied words for temperature=0.1 0.62
Copied words for temperature=0.3 0.53
Copied words for temperature=0.4 0.47
Copied words for temperature=0.6 0.41
Copied words for temperature=1.0 0.31


In [9]:
# Which datapoints are often simply copied?
# copy_traindata